Get data

In [13]:
import pandas as pd
from helpers.data_preprocessing import DataProcesser

from helpers.data_loader import DataLoader
from helpers.data_preprocessing import DataProcesser

def load_comments():
    code = DataLoader.load_csv_file("./../../data/code_data.csv", ['code'])

    def load_longer_code():
        f = open("./../../data/code.txt", "r")
        Lines = f.readlines()
        code_long = []
        for line in Lines:
            line = line.rstrip()
            # print(line)
            code_long.append(line)
        f.close()
        return code_long

    code_long = load_longer_code()
    for index, row in code.iterrows():
        curr_code_long = code_long[index]
        if curr_code_long != '':
            row['code'] = curr_code_long
    return code
code = load_comments()
code.head()

df = pd.read_csv("./../../data/train_set_0520.csv", usecols=['comment', 'non-information'])
df['code'] = code['code']
df.head()

,comment,non-information,code
0,@implNote taken from {@link com.sun.javafx.sce...,yes,"""public static void showContextMenu(TextArea t..."
1,icon.setToolTipText(printedViewModel.getLocali...,yes,"TABLE_ICONS.put(SpecialField.PRINTED, icon);"
2,Synchronize changes of the underlying date val...,no,BindingsHelper.bindBidirectional(valueProperty...
3,Ask if the user really wants to close the give...,yes,"""private boolean confirmClose(BasePanel panel)..."
4,css: information *,no,"INTEGRITY_INFO(MaterialDesignIcon.INFORMATION),"


Preprocess comments and code
1. Separate words from camel case and snake case,
2. To lower case,
3. Remove java tags and keywords,
4. remove accented chars,
5. stem

In [14]:
df['comment'] = df['comment'].apply(str)
df['comment'] = df['comment'].apply(DataProcesser.extract_camel_case)
df['comment'] = df['comment'].apply(DataProcesser.remove_stopwords)
df['comment'] = df['comment'].apply(DataProcesser.extract_snake_case)
df['comment'] = df['comment'].apply(DataProcesser.preprocess)

df['code'] = df['code'].apply(str)
df['code'] = df['code'].apply(DataProcesser.extract_camel_case)
df['code'] = df['code'].apply(DataProcesser.extract_snake_case)
df['code'] = df['code'].apply(DataProcesser.remove_java_keywords)
df['code'] = df['code'].apply(DataProcesser.preprocess_code)
df['code'] = df['code'].apply(DataProcesser.remove_stopwords)

df.head()

,comment,non-information,code
0,impl note taken link com sun javafx scene cont...,yes,show context menu text area text area context ...
1,icon set tool tip text pred view model get local,yes,tabl icon put special field pred icon
2,synchron chang underli date valu tempor access...,no,bind helper bind bidirect valu properti tempor...
3,ask user realli want close given databas,yes,confirm close base panel panel filenam panel g...
4,css inmat,no,egriti info materi design icon inmat


Count common words between code and comments

In [17]:
from helpers.textual_analysis import *

df['word_counts'] = df.apply(lambda row: count_common_words(row['comment'], row['code']), axis=1)
df.head()


,comment,non-information,code,word_counts
0,impl note taken link com sun javafx scene cont...,yes,show context menu text area text area context ...,0.036810
1,icon set tool tip text pred view model get local,yes,tabl icon put special field pred icon,0.235294
2,synchron chang underli date valu tempor access...,no,bind helper bind bidirect valu properti tempor...,0.352941
3,ask user realli want close given databas,yes,confirm close base panel panel filenam panel g...,0.044199
4,css inmat,no,egriti info materi design icon inmat,0.250000


Average of counts of common words between code and comments for non-information: yes

In [20]:
from helpers.textual_analysis import *

bad_comments = df[df['non-information'] == 'yes']
bad_comments['word_counts'].mean()


0.08412258363570375

Average of counts of common words between code and comments for non-information: no

In [21]:
from helpers.textual_analysis import *

bad_comments = df[df['non-information'] == 'no']
bad_comments['word_counts'].mean()

0.07760273940592917

Median of counts of common words between code and comments for non-information: yes

In [7]:
from helpers.textual_analysis import *

bad_comments = df[df['non-information'] == 'yes']
bad_comments['word_counts'].median()


0.0

Average of counts of common words between code and comments for non-information: no

In [8]:
from helpers.textual_analysis import *

bad_comments = df[df['non-information'] == 'no']
bad_comments['word_counts'].median()

0.008583690987124463

Average of counts of common words between code and comments for non-information: yes

In [9]:
from helpers.textual_analysis import *

bad_comments = df[df['non-information'] == 'yes']
bad_comments = bad_comments[bad_comments['word_counts'] != 0]
bad_comments['word_counts'].mean()


0.270655269088786

Average of counts of common words between code and comments for non-information: no

In [10]:
from helpers.textual_analysis import *

good_comments = df[df['non-information'] == 'no']
good_comments = good_comments[good_comments['word_counts'] != 0]
good_comments['word_counts'].mean()

0.15370992280617662

The number of no common words- between comment and code for non-information: yes

In [24]:
from helpers.textual_analysis import *

bad_comments = df[df['non-information'] == 'yes']
bad_comments_num = bad_comments.count()
bad_comments = bad_comments[bad_comments['word_counts'] == 0]
bad_comments.count()/bad_comments_num

comment            0.689189
non-information    0.689189
code               0.689189
word_counts        0.689189
dtype: float64

The number of no common words- between comment and code  for non-information: no

In [25]:
from helpers.textual_analysis import *

good_comments = df[df['non-information'] == 'no']
good_comments_num = good_comments.count()
good_comments = good_comments[good_comments['word_counts'] == 0]
good_comments.count()/good_comments_num




comment            0.495135
non-information    0.495135
code               0.495135
word_counts        0.495135
dtype: float64